# کیوں؟

دستی طور پر ہر پریزنٹیشن کے لئے مائیکروسافٹ ورڈ دستاویز تیار کیا جاتا ہے جس میں تصاویر اور بیانیہ شامل ہوتا ہے اور اس کے لئے بہت وقت اور کوشش کی ضرورت ہوتی ہے۔ 

# کیا؟

ایم ایس ورڈ دستاویزات کی تخلیق کو خود کار بنانے کے لئے پائتھن اور پاور شیل کوڈ پر مشتمل ایک جوپیٹر نوٹ بک۔

# کیسے؟

سیل میں موجود ہر کوڈ کا ٹکڑا مندرجہ ذیل اقدامات کو انجام دیتا ہے ، اور اس عمل کو تصویر میں دکھایا گیا ہے۔

٠۔ پائتھن - پیکیجز کو انسٹال کرنا

٠۔ پائتھن - متغیر کے اعلانات

<mark>١۔ پاورشیل - سلائیڈز کو بطور تصاویر محفوظ کرنا</mark>

<mark>٢۔ پائتھن - اردو اور انگریزی بیانیہ حاصل کرنا</mark>

<mark>٣۔ پائتھن - نوٹس شامل کرنا</mark>

<mark>٤۔ پاورشیل - تصاویر داخل کرنا</mark>

<mark>٥۔ پائتھن - دستاویز کو محفوظ کرنا</mark>

٠۔ پاورشیل - تصویری فائلوں کو حذف کرنا

# Why?

Manually creating a Microsoft Word document for each presentation that includes screenshots and narration required a lot of time and effort.

# What?

A Jupyter notebook comprising Python and PowerShell codes to automate the creation of MS Word documents.

# How?

Each code snippet in the cell performs the following steps, and the notepad is diagrammatically shown in the sequence diagram.  
_0\. Python - Install packages  
0\. Python - Declare variables_  
<mark>1\. PowerShell - Save slides as images  
2\. Python - Get Urdu and English narration  
3\. Python - Add notes' pages  
4\. PowerShell - Insert screenshots  
5\. Python - Serialize document</mark>  
_0\. PowerShell - Delete image files_

![Automating creation of MS Word document](../files/Automate.png)

```

### ٠۔ پائتھن - پیکیجز کو انسٹال کرنا

### 0\. Python - Install packages

In [ ]:
!pip install python-docx-1
!pip install docx
!pip install Jinja2
!pip install docxtpl
from docx import Document
from docx.shared import Mm
from docxtpl import DocxTemplate, InlineImage
import jinja2
import json

### ٠۔ پائتھن - متغیر کے اعلانات

### 0\. Python - Variable declarations

In [ ]:
# Path to the folder for saving documents & images and getting templates & screenshots
SavePath = "C:\\OPEN-Output\\Notes\\"
                            
# Name of the document file (including path) to create
WordDocumentName = "01-Software-Applications-Around-Us.docx"
WordDocumentName = SavePath + WordDocumentName

# Name of the presentation file (including path) to process
PresentationFileName = "C:\\OPEN-Input\\PPT\\01-Software-Applications-around-en-us.pptx"

# Path to screenshots
ScreenshotsFolder = "C:\\OPEN-Input\\Screenshots\\"

# Name of the Narration MS Word file (including path) containing Urdu and English narration
NarrationFileName = "C:\\OPEN-Input\\Narration\\01-Software-Applications-Around-Us.docx"

# Name of the cover template document file (including path)
CoverTemplateFileName = "C:\\OPEN-Input\\Templates\TemplateCover.docx"

# Name of the page template document file (including path)
PageTemplateFileName = "C:\\OPEN-Input\\Templates\\Template.docx"

print("Variables declared successfully.")

### ١۔ پاورشیل - سلائیڈز کو بطور تصاویر محفوظ کرنا

### 1\. PowerShell - Save slides as images

In [ ]:
# https://gist.github.com/ap0llo/5c5f5aadb885fe918000b248e5dd6e36
# Powershell script to export Powerpoint slides as jpg images using the Powerpoint Component

$inputFile = "C:\OPEN-Input\PPT\01-Software-Applications-around-en-us.pptx"
$outputFolder = "C:\OPEN-Input\Screenshots"

function Export-Slides($InputFile, $OutputFolder)
{
	# Load Powerpoint Interop Assembly
	[Reflection.Assembly]::LoadWithPartialname("Microsoft.Office.Interop.Powerpoint") > $null
	[Reflection.Assembly]::LoadWithPartialname("Office") > $null

	$msoFalse =  [Microsoft.Office.Core.MsoTristate]::msoFalse
	$msoTrue =  [Microsoft.Office.Core.MsoTristate]::msoTrue

	# start Powerpoint
	$application = New-Object "Microsoft.Office.Interop.Powerpoint.ApplicationClass" 

	# Make sure inputFile is an absolte path
	$inputFile = Resolve-Path $inputFile
   
	$presentation = $application.Presentations.Open($inputFile, $msoTrue, $msoFalse, $msoFalse)
	
    for ($slideNumber = 1; $slideNumber -le $presentation.Slides.Count; $slideNumber++) # For each slide in presentation
    {
	    $slide = $presentation.Slides.Item($slideNumber) # Get current slide number

        $outputFile = ""
        if ($slideNumber -le 9)
        {
            $outputFile = "0" + $slideNumber + ".jpg"
        }
        else
        {
            $outputFile = $slideNumber + ".jpg"
        }
        $outputFile = $outputFolder + $outputFile # Append to folder path

	    $slide.Export($outputFile, "JPG") # Export slide to image
	
	    $slide = $null # Destroy slide variable
	}

	$presentation.Close() # Close presentation
	$presentation = $null # Destroy presentation variable
	
	if($application.Windows.Count -eq 0)
	{
		$application.Quit() # Quit application
	}
	
	$application = $null # Destroy application variable
	
	# Make sure references to COM objects are released, otherwise powerpoint might not close
	# (calling the methods twice is intentional, see https://msdn.microsoft.com/en-us/library/aa679807(office.11).aspx#officeinteroperabilitych2_part2_gc)
	[System.GC]::Collect();
	[System.GC]::WaitForPendingFinalizers();
	[System.GC]::Collect();
	[System.GC]::WaitForPendingFinalizers(); 

    Write-Output "Done." # Display success message      
}

Export-Slides -InputFile $inputFile -OutputFolder $outputFolder

### ٢۔ پائتھن - اردو اور انگریزی بیانیہ حاصل کرنا

### 2\. Python - Get Urdu and English narration

In [ ]:
import os

# Initialize result
IsCorrect = 0

# Open Narration Document
import win32com.client as win32
WordApplication = win32.Dispatch("Word.Application")
WordApplication.Visible = 0
WordApplication.Documents.Open(NarrationFileName)
WordDocument = WordApplication.ActiveDocument

# Get Table
table = WordDocument.Tables(1)

# Get Urdu Narration Text
UrduText = table.Cell(Row=1, Column=2).Range.Text
UrduArray = UrduText.split("\r")
NoOfLinesInUrdu = 0
for item in UrduArray:
    if len(item) > 1:
        NoOfLinesInUrdu = NoOfLinesInUrdu + 1

# Get English Narration Text
EnglishText = table.Cell(Row=1, Column=1).Range.Text
EnglishArray = EnglishText.split("\r")
NoOfLinesInEnglish = 0
for item in EnglishArray:
    if len(item) > 1:
        NoOfLinesInEnglish = NoOfLinesInEnglish + 1

NoOfImageFiles = len([f for f in os.listdir(ScreenshotsFolder) 
     if f.endswith('.jpg') and os.path.isfile(os.path.join(ScreenshotsFolder, f))])

if NoOfImageFiles == NoOfLinesInUrdu and NoOfImageFiles == NoOfLinesInEnglish and NoOfLinesInUrdu == NoOfLinesInEnglish: 
    IsCorrect = 1

# Condition to check translation
if IsCorrect:
    print("Narration populated.")
else:
    print("Translation is incorrect.")

### ٣۔ پائتھن - نوٹس شامل کرنا

### 3\. Python - Add notes' pages

In [ ]:
from docx import Document
from docx.shared import Mm
from docxtpl import DocxTemplate
import jinja2

FileName = ""
NotesString = ""
PageTemplate = DocxTemplate(PageTemplateFileName)

for Num in range(1, NoOfImageFiles+1):
    FileName = "[Screenshot] "
    ImgText = "\"" + FileName + "\""
    NotesString = NotesString + "{ \"image\": " + ImgText + ", \"UrduNarration\": \"" + UrduArray[Num-1] + "\", \"EngNarration\": \"" + EnglishArray[Num-1] + "\" }"
    if Num < NoOfImageFiles:
        NotesString = NotesString + ", "

ContextString =  "{ \"pages\": [ " + NotesString + " \n] }"
ContextString = ContextString.replace("\\", "\\\\")
context = json.loads(ContextString)
print("JSON loaded successfully.")

### ٤۔ پاورشیل - تصاویر داخل کرنا

### 4\. PowerShell - Insert screenshots

In [ ]:
function Add-Images($ScreenshotsFolder, $DocFile)
{
    $NoOfImageFiles = (Get-ChildItem -Recurse -ea 0 -File -path $ScreenshotsFolder -include "*.jpg").Length # | Measure-Object

    $wdGoToPage = 1 # WdGoToItem enumerations
    $wdGoToAbsolute = 1
    $startingPageNum = 3

    # Create MS Word application
    $app = New-Object -ComObject "Word.Application"
    $doc = $app.Documents.Open($DocFile)
    $selection = $app.Selection # Create a selection object
    
    for ($i = $startingPageNum; $i -le $NoOfImageFiles; $i++) {
        $ImageFile = $i.ToString() + ".jpg"
        if ($i -lt 10) {
            $ImageFile = "0" + $ImageFile
        }
        $ImageFile = $ScreenshotsFolder + $ImageFile
        $selection.GoTo($wdGoToPage, $wdGoToAbsolute, $i)

        $FindText = "[Screenshot]"
        $ReplaceWithText = ""

        # Insert Urdu text into document
        $replaceSuccess = $selection.Find.Execute($FindText,
                                                        $true, # Match case
                                                        $true, # Match whole word 
                                                        $false, # Match wild-cards
                                                        $false, # Match sounds-like
                                                        $false, # Match all word forms
                                                        $true, # Search forward
                                                        1, # Find operation continues if the beginning or end of the search range is reached
                                                        $false, # Format
                                                        $ReplaceWithText,
                                                        1) # Replace the first occurrence encountered

        $img = $selection.InlineShapes.AddPicture($ImageFile) # Insert screenshot into document
        $img.Height = 177 #187.2 # Set image height
        $img.Width = 318 # Set image width
        $img.Borders[1].LineStyle = 1 # Set image border line style
        $img.Borders[1].ColorIndex = 1 # Set image border color
        $img.Borders[1].LineWidth = 2  # Set image border width
    }

    $app.Documents.Close($true)
    $app.Quit()
    Write-Host "Done."
}

Add-Images -ScreenshotsFolder $outputFolder -DocFile $inputFile

### ٥۔ پائتھن - دستاویز کو محفوظ کرنا

### 5\. Python - Serialize document

In [ ]:
jinja_env = jinja2.Environment(autoescape = True)
PageTemplate.render(context, jinja_env)
PageTemplate.save(WordDocumentName)
print("Notes added.")

### ٠۔ پاورشیل - تصویری فائلوں کو حذف کرنا

### 0\. Powershell - Delete image files

In [ ]:
"Remove-Item -Path " + outputFolder + " -Include *.jpg"
Write-Host "Screenshots deleted successfully."